In [1]:
from pyspark.sql import SparkSession

## *****PySpark DF Question*****
## 1) Find count of CLOSED, PENDING_PAYMENT, ON_HOLD & COMPLETE orders?
## Filename: orders.txt

In [2]:
spark = SparkSession.builder.appName("learning").master("local").getOrCreate()
sc = spark.sparkContext

In [7]:
orders_df = spark.read.format("csv").option("header",'True').option("delimiter",",").option("inferSchema",'True')\
             .load("file:///home/saif/LFS/datasets/orders.txt")

orders_df.printSchema()
orders_df.show(5)

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
+--------+--------------------+-----------------+---------------+
only showing top 5 rows



In [9]:
orders_df.where(
         (orders_df.order_status == 'CLOSED')|
         (orders_df.order_status == 'PENDING_PAYMENT')|
         (orders_df.order_status == 'ON_HOLD') | 
         (orders_df.order_status == 'COMPLETE'))\
         .groupby("order_status").count().show()

+---------------+-----+
|   order_status|count|
+---------------+-----+
|PENDING_PAYMENT|15030|
|       COMPLETE|22899|
|        ON_HOLD| 3798|
|         CLOSED| 7556|
+---------------+-----+



## *****PySpark DF Question*****
## 1) Input: Given data from 1 to 10 perform below activities. Output: Find Odd & Even Records & save them in different files & restrict data below to 6 & save it another file. Perform this using DSL 
## & SparkSQL and save the results to location 
## /user/saif/HFS/CCA_175/Output/df_op & ~/sql_op.

In [54]:
data = [1,2,3,4,5,6,7,8,9,10]
df = spark.createDataFrame(data,'string')
df.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
|    8|
|    9|
|   10|
+-----+



In [57]:
even_df = df.where(df.value%2 == 0)
even_df.show()

odd_df = df.where(df.value%2 != 0)
odd_df.show()

below_six_df = df.where(df.value < 6)
below_six_df.show()

+-----+
|value|
+-----+
|    2|
|    4|
|    6|
|    8|
|   10|
+-----+

+-----+
|value|
+-----+
|    1|
|    3|
|    5|
|    7|
|    9|
+-----+

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
+-----+



In [63]:
below_six_df.write.format('csv')\
                 .mode('overwrite')\
                 .save('hdfs://localhost:9000/user/saif/HFS/Output/df_op/df_below_six')

In [85]:
from pyspark.sql.types import StructType, StringType, IntegerType,StructField

In [90]:
# Why Am i getting error Still yet to learn

deptdata = [(1000),
            (20000),
            (30000),
            (400000)]

data_schema = StructType([
                StructField("deptno", IntegerType(), True)
               ])

df = spark.createDataFrame(data=deptdata, schema = data_schema) 
df.printSchema()
df.show(truncate=False) 

## Error  => Can not infer schema for type: <class 'int'>

TypeError: StructType can not accept object 1000 in type <class 'int'>

In [94]:
data = [("James","","Smith","36","M",3000),    
        ("Michael","Rose","","40","M",4000),    
        ("Robert","","Williams","42","M",4000),    
        ("Maria","Anne","Jones","39","F",4000),    
        ("Jen","Mary","Brown","","F",-1)  
       ]
schema = StructType([     
    StructField("firstname",StringType(),True),     
    StructField("middlename",StringType(),True),     
    StructField("lastname",StringType(),True),     
    StructField("age", StringType(), True),     
    StructField("gender", StringType(), True),     
    StructField("salary", IntegerType(), True)   ]) 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+---+------+------+
|firstname|middlename|lastname|age|gender|salary|
+---------+----------+--------+---+------+------+
|James    |          |Smith   |36 |M     |3000  |
|Michael  |Rose      |        |40 |M     |4000  |
|Robert   |          |Williams|42 |M     |4000  |
|Maria    |Anne      |Jones   |39 |F     |4000  |
|Jen      |Mary      |Brown   |   |F     |-1    |
+---------+----------+--------+---+------+------+



## 2)
## Input: 
## You are provided with two files products.txt & suppliers.txt at location 
## /home/saif/LFS/cca175/. 
## 
## Output: 
## Select product, its price, its supplier name where product price is less than 0.6 & write the output in parquet format to location /user/saif/HFS/CCA_175/Output.

In [7]:
products_df = spark.read.format("csv")\
                        .option("delimeter","|")\
                        .option("inferSchema",'True')\
                        .option('header','True')\
                        .load('file:///home/saif/LFS/datasets/products.txt')

products_df.printSchema()
products_df.show(5)

suppliers_df = spark.read.format("csv")\
                        .option("delimeter","|")\
                        .option("inferSchema",'True')\
                        .option('header','True')\
                        .load('file:///home/saif/LFS/datasets/suppliers.txt')

suppliers_df.printSchema()
suppliers_df.show(5)

root
 |-- productid: integer (nullable = true)
 |-- productcode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- supplierid: integer (nullable = true)

+---------+-----------+---------+--------+-----+----------+
|productid|productcode|     name|quantity|price|supplierid|
+---------+-----------+---------+--------+-----+----------+
|     1001|        PEN|  Pen Red|    5000| 1.23|       501|
|     1002|        PEN| Pen Blue|    8000| 1.25|       501|
|     1003|        PEN|Pen Black|    2000| 1.25|       501|
|     1004|        PEC|Pencil 2B|   10000| 0.48|       502|
|     1005|        PEC|Pencil 2H|    8000| 0.49|       502|
+---------+-----------+---------+--------+-----+----------+
only showing top 5 rows

root
 |-- supplierid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: integer (nullable = true)

+----------+-----------+----------+
|supplierid|       name|    

In [10]:
condition = (products_df["supplierid"] == suppliers_df["supplierid"])
result_df = products_df.join(suppliers_df,condition, 'inner')\
                       .where(products_df["price"] < 0.6)\
            .select(products_df["productid"],products_df["price"],suppliers_df["name"].alias("supplier_name"))

result_df.show(5)

+---------+-----+-------------+
|productid|price|supplier_name|
+---------+-----+-------------+
|     1004| 0.48|  XYZ Company|
|     1005| 0.49|  XYZ Company|
|     2001| 0.52|  ABC Traders|
|     2004| 0.47|  XYZ Company|
+---------+-----+-------------+



In [11]:
result_df.write.mode('overwrite').parquet("hdfs://localhost:9000/user/saif/HFS/Output/df_op/product_data")